In [ ]:
import pandas as pd
import numpy as np
import re, nltk, spacy, gensim
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
%matplotlib inline
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
from nltk.collocations import *

In [ ]:
import tensorflow as tf

In [ ]:
#standard functions
#function to clean text
def CleanTextColumn(TextCol, removelist=[]):
    """takes a list of text, and a list of words to clean out
    
    Returns the text column as a list with emails, new line characters, the words listed, 
    special characters, and single quotes, making all characters lower case"""
    
    # Convert to list if it's a pandas series
    if type(TextCol) == type(pd.Series()):
        TextCol = raw_df.astype(str).values.tolist()
    
    # Remove Emails
    TextCol = [re.sub('\S*@\S*\s?', '', sent) for sent in TextCol]
    
    #Remove double spaces
    TextCol = [re.sub('  ', ' ', sent) for sent in TextCol]
    
    # Remove new line characters
    TextCol = [re.sub('\s+', ' ', sent) for sent in TextCol]
    
    # Convert data to lowercase
    TextCol = [sent.lower() for sent in TextCol]
    
    # Remove single quotes
    TextCol = [re.sub("\'", "", sent) for sent in TextCol]
    
    for junk in removelist:
        TextCol = [re.sub(junk, "", sent) for sent in TextCol]
    
    # Removing special characters
    TextCol = [re.sub('[^A-Za-z ]+', '', sent) for sent in TextCol]
    
    return TextCol

#ignored_words = nltk.corpus.stopwords.words('english')
#finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in ignored_words)

In [ ]:
filepath.head()
NASB = pd.read_excel(filepath)

In [ ]:
NASB.head()

## data preparation for word tree

In [140]:
def get_WordTree_data_from_Series(series,word,n):
    """iterates over a pandas text series and creates a 
    WordTree dataset of n words before and after each instance of word"""
    wordtreedata = [['Phrases']]

    for index, value in series.items():
        wordtreedata.extend(get_WordTree_data(value,word,n))
        
    return wordtreedata

def get_WordTree_data(text,word,n):
    """takes a text, a target word, and the number of words around it desired (n) and returns data prepared for WordTree"""
    
    #convert text to list of sequential words
    text = remove_non_text(text)
    textlist = text.split()
    
    #iterate over each instance of the word in the data and append its surrounding n words to the data
    wordtreedata=[]
    word_instances = get_word_instances(textlist,word)
    
    for instance in word_instances:
        wordtreedata.append(get_words_before_after(textlist, instance, n))
    return wordtreedata

def remove_non_text(text):
    """removes doubled white space and non-alphabetic characters"""
    text = re.sub(r'([^\s\w]|_|[1-9])+', '', text) #remove non alphanumeric/whitespace
    text = re.sub(" +", " ", text) #remove white space
    return text

def get_word_instances(textlist,word):
    """returns word index of each """
    return [i for i, w in enumerate(textlist) if w == word]

def get_words_before_after(textlist,index,n):
    """takes a list of words and retrieves the n words before and after the word at index"""
    
    #initialize starting and finishing indices
    low = index-n
    high = index+n+1
    
    #filter to the last word or first word if n extends beyond the length or start of the text
    if index - n < 0:
        low = 0
    elif index+n+1 >= len(textlist):
        high = len(textlist)-1
    
    return [" ".join(textlist[low:high])]

In [127]:
# -*- coding: utf-8 -*-
"""
Created on Thu Nov 14 10:56:54 2019

@author: v-mifees
"""

#imports
import pandas as pd
import re
import dash
from dash_google_charts import WordTree

text_data = pd.read_csv("C:\\Users\\v-mifees\\Desktop\\ocvpp_frown_filtered.csv")['text']

#functions to setup data for word tree
def get_WordTree_data_from_Series(series,word,n):
    """iterates over a pandas text series and creates a 
    WordTree dataset of n words before and after each instance of word"""
    wordtreedata = [['Phrases']]

    for index, value in series.items():
        wordtreedata.extend(get_WordTree_data(value,word,n))
        
    return wordtreedata

def get_WordTree_data(text,word,n):
    """takes a text, a target word, and the number of words around it desired (n) and returns data prepared for WordTree"""
    
    #convert text to list of sequential words
    text = remove_non_text(text)
    textlist = text.split()
    
    #iterate over each instance of the word in the data and append its surrounding n words to the data
    wordtreedata=[]
    word_instances = get_word_instances(textlist,word)
    
    for instance in word_instances:
        wordtreedata.append(get_words_before_after(textlist, instance, n))
    return wordtreedata

def remove_non_text(text):
    """removes doubled white space and non-alphabetic characters"""
    text = re.sub(r'([^\s\w]|_|[1-9])+', '', text) #remove non alphanumeric/whitespace
    text = re.sub(" +", " ", text) #remove white space
    return text

def get_word_instances(textlist,word):
    """returns word index of each """
    return [i for i, w in enumerate(textlist) if w == word]

def get_words_before_after(textlist,index,n):
    """takes a list of words and retrieves the n words before and after the word at index"""
    
    #initialize starting and finishing indices
    low = index-n
    high = index+n+1
    
    #filter to the last word or first word if n extends beyond the length or start of the text
    if index - n < 0:
        low = 0
    elif index+n+1 >= len(textlist):
        high = len(textlist)-1
    
    return [" ".join(textlist[low:high])]

#Word Tree settings
series = text_data
word = 'slow'
n = 5

#initializing word tree
app = dash.Dash()

app.layout = WordTree(
    data=get_WordTree_data_from_Series(series,word,n),
    options={"wordtree": {"format": "implicit", "word": word, 'type': 'double'}},
)

if __name__ == "__main__":
    app.run_server()

[['slow', 'load', 'poor', 'pics', 'see', 'google'],
 ['slow'],
 ['so', 'slow'],
 ['Too', 'slow', 'to', 'load'],
 ['Powerpoint', 'is', 'so', 'slow'],
 ['very', 'very', 'slow'],
 ['this', 'is', 'so', 'slow'],
 ['to', 'slow'],
 ['Its', 'way', 'too', 'slow', 'Please', 'fix', 'the', 'speed'],
 ['Not', 'enought', 'options', 'very', 'slow', 'to', 'load'],
 ['loading', 'slow'],
 ['Very', 'slow', 'connecting', 'to', 'server'],
 ['Too', 'slow', 'search', 'engine'],
 ['slow'],
 ['too', 'slow'],
 ['too', 'slow'],
 ['extremly', 'slow'],
 ['The',
  'speed',
  'is',
  'excruciatingly',
  'slow',
  'even',
  'though',
  'the',
  'internet',
  'is'],
 ['very', 'slow', 'to', 'load', 'the', 'library'],
 ['do',
  'it',
  'everything',
  'is',
  'really',
  'slow',
  'I',
  'really',
  'dislike',
  'this',
  'because'],
 ['but',
  'its',
  'just',
  'PowerPoint',
  'thats',
  'slow',
  'Please',
  'fix',
  'this',
  'error',
  'It'],
 ['to', 'slow'],
 ['slide',
  'and',
  'it',
  'is',
  'extremely',
  'sl

In [ ]:
def graph_topwords(text):
    # Creating a tokenizer
    tokenizer = nltk.tokenize.RegexpTokenizer('\w+')

    # Tokenizing the text
    tokens = tokenizer.tokenize(text)

    #get all as lowercase
    words = [w.lower() for w in tokens]

    nltk.download('stopwords')

    sw = nltk.corpus.stopwords.words('english')

    words_ns = []

    for word in words:
        if word not in sw:
            words_ns.append(word)

    # This command display figures inline
    %matplotlib inline

    # Creating the word frequency distribution
    freqdist = nltk.FreqDist(words_ns)

    freqdist.xlabel = 'Word'

    # Plotting the word frequency distribution
    freqdist.plot(25)

graph_topwords(alltext)

In [ ]:
# Import the wordcloud library
from wordcloud import WordCloud

# Join the different processed titles together.
long_string = positivetext

# Create a WordCloud object
wordcloud = WordCloud()

# Generate a word cloud
wordcloud.generate(long_string)

# Visualize the word cloud
wordcloud.to_image()

## Collocations

In [ ]:
#put text here
text = 'this that then some'

#tokenize text
tokens = nltk.word_tokenize(text)

#collocations
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = BigramCollocationFinder.from_words(tokens)
numngrams = 10
raw_collocations_bi = finder.nbest(bigram_measures.pmi, numngrams)

#applying frequency filter
min_occurences = 3
finder.apply_freq_filter(min_occurences)
filt_collocations_bi = finder.nbest(bigram_measures.pmi, numngrams)

#trigrams
finder = TrigramCollocationFinder.from_words(tokens)
scored = finder.score_ngrams(trigram_measures.raw_freq)

#top n results
sorted(finder.nbest(trigram_measures.raw_freq, 2))

#spanning intervening words
finder = TrigramCollocationFinder.from_words(tokens, window_size=4)
sorted(finder.nbest(trigram_measures.raw_freq, 4))

## Appendix

In [ ]:
#code in waiting
#test cell
import re
[m.start() for m in re.finditer('test', 'test test test test')]

from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

similar("love","loves")